download_lila_subset.py

Example of how to download a list of files from LILA, e.g. all the files
in a data set corresponding to a particular species.

So i started with the code given in the repo: https://github.com/agentmorris/MegaDetector/blob/main/megadetector/data_management/lila/lila_common.py

and then realized their functions here were for machines that can have variables and files bigger than 20gb atleast in their RAM+Virtual ram, so the metadataset file downloaded and unzipped with chunks and concatenated them, however i can't open up the csv file for the metadata as it is around 15.5gb and my virtual memory and ram fill up even with most of the applications closed, The dataset and the metadata is simply too big so then i tried editing the given code to filter out the required columns and data (snow leopards) but that query doesnt show results as shown in the code blocks below, so im trying to debug what the scientific name for the snow leopards is without looking at the metadata, ill look for something on the web until then

In [1]:
# pip install -r MegaDetector/envs/requirements.txt

In [2]:
import os
import random
import sys
from tqdm import tqdm
from collections import defaultdict

sys.path.append('MegaDetector')

from megadetector.data_management.lila.lila_common import \
    read_lila_all_images_file, is_empty, lila_base_urls

for s in lila_base_urls.values():
    assert s.endswith('/')

In [3]:
import importlib
import sys
# Find and remove the module from sys.modules
if 'megadetector.data_management.lila.lila_common' in sys.modules:
    del sys.modules['megadetector.data_management.lila.lila_common']
# Or if you're importing it differently, use that import path

# Now import it again
from megadetector.data_management.lila.lila_common import read_lila_all_images_file

In [4]:
# If any of these strings appear in the common name of a species, we'll download that image
# species_of_interest = ['grey fox','gray fox','cape fox','red fox','kit fox']
species_of_interest = ['leopard','snow leopard','leopardus']

# We'll write images, metadata downloads, and temporary files here
lila_local_base = 'D:/LilaDataset'

metadata_dir = os.path.join(lila_local_base,'metadata')
os.makedirs(metadata_dir,exist_ok=True)

output_dir = os.path.join(lila_local_base,'lila_downloads_by_dataset')
os.makedirs(output_dir,exist_ok=True)

# Number of concurrent download threads
n_download_threads = 20

max_images_per_dataset = 2000 # None

preferred_provider = 'aws' # 'azure', 'gcp', 'aws'

random.seed(0)

In [5]:
# import pandas as pd
# import os
# from collections import defaultdict

# def custom_read_lila_all_images_file(metadata_dir):
#     """
#     Custom version of read_lila_all_images_file that handles large files better
#     """
#     unzipped_csv_filename = os.path.join(metadata_dir, 'lila-all-images.csv')
    
#     if os.path.exists(unzipped_csv_filename):
#         print(f'Reading {unzipped_csv_filename} in chunks...')
#         # Read in chunks to avoid memory issues
#         chunks = pd.read_csv(unzipped_csv_filename, low_memory=True, chunksize=50000)
#         df = pd.concat([chunk for chunk in chunks])
#         return df
#     else:
#         print(f"Error: File {unzipped_csv_filename} not found")
#         return None

In [6]:
#%% Download and open the giant table of image URLs and labels

# Takes ~2 minutes to download, unzip, and open
df = read_lila_all_images_file(metadata_dir)


Bypassing download of already-downloaded file lila_image_urls_and_labels.csv.zip
D:/LilaDataset\metadata\lila_image_urls_and_labels.csv already unzipped


c:\Courses Junior Spring\Deep Learning\project\MegaDetector\megadetector\data_management\lila\lila_common.py:214: DtypeWarning: Columns (11,23,24,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([chunk for chunk in df])
c:\Courses Junior Spring\Deep Learning\project\MegaDetector\megadetector\data_management\lila\lila_common.py:214: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([chunk for chunk in df])
c:\Courses Junior Spring\Deep Learning\project\MegaDetector\megadetector\data_management\lila\lila_common.py:214: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([chunk for chunk in df])
c:\Courses Junior Spring\Deep Learning\project\MegaDetector\megadetector\data_management\lila\lila_common.py:214: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=

In [7]:
#%% Find all the images we want to download

# Takes ~2 minutes

common_name_to_count = defaultdict(int)

ds_name_to_urls = defaultdict(list)

def find_items(row):
    
    if is_empty(row['common_name']):
        return
    
    match = False
    
    # This is the only bit of this file that's specific to a particular query.  In this case
    # we're checking whether each row is on a list of species of interest, but you do you.
    for species_name in species_of_interest:
        if species_name in row['common_name']:
            match = True
            common_name_to_count[species_name] += 1
            break
    
    if match:
        ds_name_to_urls[row['dataset_name']].append(row['url_' + preferred_provider])

tqdm.pandas()
_ = df.progress_apply(find_items,axis=1)

# We have a list of URLs for each dataset, flatten them all into a list of URLs
all_urls = list(ds_name_to_urls.values())
all_urls = [item for sublist in all_urls for item in sublist]
print('Found {} matching URLs across {} datasets'.format(len(all_urls),len(ds_name_to_urls)))

for common_name in common_name_to_count:
    print('{}: {}'.format(common_name,common_name_to_count[common_name]))
    
from copy import deepcopy
ds_name_to_urls_raw = deepcopy(ds_name_to_urls)


#%% Optionally trim to a fixed number of URLs per dataset

  0%|          | 0/23723546 [00:00<?, ?it/s]

100%|██████████| 23723546/23723546 [29:56<00:00, 13208.81it/s] 


Found 3997 matching URLs across 10 datasets
leopard: 3997


In [8]:
print(df.head(100))

            dataset_name                                            url_gcp  \
0   Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
1   Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
2   Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
3   Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
4   Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
..                   ...                                                ...   
95  Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
96  Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
97  Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
98  Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   
99  Caltech Camera Traps  https://storage.googleapis.com/public-datasets...   

                                              url_a

In [9]:
if max_images_per_dataset is None:
    pass
else:
    # ds_name = next(iter(ds_name_to_urls.keys()))
    for ds_name in ds_name_to_urls:
        if len(ds_name_to_urls[ds_name]) > max_images_per_dataset:
            ds_name_to_urls[ds_name] = random.sample(ds_name_to_urls[ds_name],max_images_per_dataset)


#%% Choose target files for each URL

from megadetector.data_management.lila.lila_common import lila_base_urls

# We have a list of URLs per dataset, flatten that into a single list of URLs
urls_to_download = set()
for ds_name in ds_name_to_urls:
    for url in ds_name_to_urls[ds_name]:
        urls_to_download.add(url)
urls_to_download = sorted(list(urls_to_download))        

# A URL might look like this:
#
# https://storage.googleapis.com/public-datasets-lila/wcs-unzipped/animals/0667/0302.jpg
# 
# We'll write that to an output file that looks like this (relative to output_dir):
#
# wcs-unzipped/animals/0667/0302.jpg
#
# ...so we need to remove the base URL to get the target file.
base_url = lila_base_urls[preferred_provider]
assert base_url.endswith('/')

url_to_target_file = {}

for url in urls_to_download:
    assert url.startswith(base_url) 
    target_fn_relative = url.replace(base_url,'')
    target_fn_abs = os.path.join(output_dir,target_fn_relative)
    url_to_target_file[url] = target_fn_abs


#%% Download image files

from megadetector.utils.url_utils import parallel_download_urls

download_results = parallel_download_urls(url_to_target_file=url_to_target_file,
                                          verbose=False,
                                          overwrite=False,
                                          n_workers=n_download_threads,
                                          pool_type='thread')


#%% Open output folder

from megadetector.utils.path_utils import open_file
open_file(output_dir)


#%% Scrap

if False:
    
    pass

    #%% Find all the reptiles on LILA

    reptile_rows = df.loc[df['class'] == 'reptilia']
    
    # i_row = 0; row = reptile_rows.iloc[i_row]
    
    common_name_to_count = defaultdict(int)
    dataset_to_count = defaultdict(int)
    for i_row,row in reptile_rows.iterrows():
        common_name_to_count[row['common_name']] += 1
        dataset_to_count[row['dataset_name']] += 1
        
    from megadetector.utils.ct_utils import sort_dictionary_by_value
    
    print('Found {} reptiles\n'.format(len(reptile_rows)))
    
    common_name_to_count = sort_dictionary_by_value(common_name_to_count,reverse=True)
    dataset_to_count = sort_dictionary_by_value(dataset_to_count,reverse=True)
    
    print('Common names by count:\n')
    for k in common_name_to_count:
        print('{} ({})'.format(k,common_name_to_count[k]))
    
    print('\nDatasets by count:\n')    
    for k in dataset_to_count:
        print('{} ({})'.format(k,dataset_to_count[k]))

Preparing download list


100%|██████████| 3997/3997 [00:00<00:00, 16879.18it/s]


Starting a thread pool with 20 workers


 15%|█▌        | 617/3997 [02:36<19:12,  2.93it/s] 

 18%|█▊        | 734/3997 [03:46<25:45,  2.11it/s]

100%|██████████| 3997/3997 [18:16<00:00,  3.65it/s] 
